# 05. Plugin: Math Function
This tutorial shows you how to create plugins with Semantic Kernel.

## Prerequisites

Refer to [01-get-started.ipynb](./01-get-started.ipynb) to install Ollama.

## Install Semantic Kernel

Run the following cell to install the packages.

In [1]:
#r "nuget: Microsoft.SemanticKernel, 1.11.1"

Installed Packages Microsoft.SemanticKernel, 1.11.1

## Instantiate the kernel

In [15]:
#pragma warning disable SKEXP0010
using Microsoft.SemanticKernel;
using Kernel = Microsoft.SemanticKernel.Kernel;

Action<string> println = System.Console.WriteLine;
Action<string> print = System.Console.Write;

// We use Mistral for this tutorial 
var modelId = "mistral";
// local Ollama endpoint
var endpoint = new Uri("http://localhost:11434");

var kernelBuilder = Kernel.CreateBuilder();
var kernel = kernelBuilder
    .AddOpenAIChatCompletion(
        modelId,
        endpoint,
        apiKey:null) 
    .Build();

#pragma warning restore SKEXP0010

##  Create the plugin

Let's first create a Math plugin with four functions `Add`, `Subtract`, `Multiply`, and `Divide`.


In [16]:
using System.ComponentModel;
using Microsoft.SemanticKernel;

public sealed class MyMathPlugin
{

    [KernelFunction, Description("Add two integers")]
    public static int Add(
        [Description("The first integer to add")] int number1,
        [Description("The second integer to add")] int number2
    )
    {
        return number1 + number2;
    }

    [KernelFunction, Description("Subtract two integers")]
    public static int Subtract(
        [Description("The first integer to subtract from")] int number1,
        [Description("The second integer to subtract away")] int number2
    )
    {
        return number1 - number2;
    }

    [KernelFunction, Description("Multiply two integers.")]
    public static int Multiply(
        [Description("The first integer to multiply")] int number1,
        [Description("The second integer to multiply")] int number2
    )
    {
        return number1 * number2;
    }

    [KernelFunction, Description("Divide two integers. Make sure the second integer is not 0.")]
    public static int Divide(
        [Description("The first integer to multiply")] int number1,
        [Description("The second integer to multiply")] int number2
    )
    {
        return number1 / number2;
    }
    
}

## Register the plugin with the kernel

After adding the plugin, we can then call it natively first.

In [23]:
public void PrintPlugins()
{
    foreach(var p in kernel.Plugins)
    println(p.Name);
}

In [26]:
if(kernel.Plugins.Count == 0)
{
    kernel.Plugins.AddFromType<MyMathPlugin>();
}

PrintPlugins();

KernelArguments args = new() {
    ["number1"]=10, 
    ["number2"]=20
};
var result = await kernel.InvokeAsync<int>("MyMathPlugin","Add", args);
Console.WriteLine(result);

MyMathPlugin
30


## Try the plugin in Prompts

Now let's allow AI to call the plugin/functions/tools directly. Pay attention to `ToolCallBehavior` setting of `OpenAIPromptExecutionSettings`.

In [57]:
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.SemanticKernel.TemplateEngine;

var chatService = kernel.GetRequiredService<IChatCompletionService>();

var executionSettings = new OpenAIPromptExecutionSettings
{
    MaxTokens = 2000,
    Temperature = 0,
    // Enable automatic invocation of kernel functions
    ToolCallBehavior = ToolCallBehavior.AutoInvokeKernelFunctions
};

var skPrompt = "What is 10 plus 20?";

// Let's print out the prompt
var promptTemplateConfig = new PromptTemplateConfig(skPrompt);
var promptTemplateFactory = new KernelPromptTemplateFactory();
var promptTemplate = promptTemplateFactory.Create(promptTemplateConfig);
print(await promptTemplate.RenderAsync(kernel));


What is 10 plus 20?

In [63]:
List<string> questions = [skPrompt];
// Create chat history
ChatHistory history = [];
foreach(var question in questions)
{
    history.AddUserMessage(question);
    println($"User > {question}");
    var result = chatService.GetStreamingChatMessageContentsAsync(
                                history,
                                executionSettings: executionSettings,
                                kernel: kernel);

    // Stream the results
    string fullMessage = "";
    var first = true;
    await foreach (var content in result)
    {
        if (content.Role.HasValue && first)
        {
            print("Assistant > ");
            first = false;
        }
        print(content.Content);
        fullMessage += content.Content;
    }

    // Add the message from the agent to the chat history
    history.AddAssistantMessage(fullMessage);

    // Another approach: Get the response not in streaming mode
    // var response = await chatService.GetChatMessageContentAsync(question, executionSettings, kernel: kernel);
    // response.Display();
    // println(response.Content);
}

User > What is 10 plus 20?
Assistant >  The sum of 10 and 20 is 30. In mathematical terms, you can represent this as: 10 + 20 = 30.